# Pre-Equilibrium Maximazation of Yield #

In [1]:
# make sure jupyter path is correct for loading local moudules
import sys
# path to steric_simulator module relative to notebook
sys.path.append("../../")
import copy

In [2]:
from steric_free_simulator import ReactionNetwork, VectorizedRxnNet, VecSim, Optimizer, EquilibriumSolver

EnergyExplorer Module is not available. Check Rosetta installation. <ipykernel.iostream.OutStream object at 0x7f4fa3641e80>


We'll start with the AP2 complex that we've worked with before. Pairwise $\Delta Gs$ were derived from the PDB structures via Rossetta

In [3]:
base_input = '../input_files/trimer.pwr'
rn = ReactionNetwork(base_input, one_step=True)
rn.resolve_tree()


['A']
['B']
['C']
-----
{'A'}
{'A'}
set()
-----
{'A'}
{'B'}
{'A'}
False
Allowed edges: 
('A', 'B')
Connected Nodes:  ['A', 'B']
Connected Edges:  [('A', 'B')]
Allowed edges: 
('A', 'C')
Allowed edges: 
('B', 'C')
New node added--1
['A', 'B']
{('A', 'B'): [None, None, 1, tensor([-50000.], dtype=torch.float64)], ('A', 'C'): [None, None, 1, tensor([-50000.], dtype=torch.float64)], ('B', 'C'): [None, None, 1, tensor([-50000.], dtype=torch.float64)]}
New node added--2
[0, 1, 2, 3]
-----
{'A'}
{'C'}
{'A'}
False
Allowed edges: 
('A', 'B')
Allowed edges: 
('A', 'C')
Connected Nodes:  ['A', 'C']
Connected Edges:  [('A', 'C')]
Allowed edges: 
('B', 'C')
New node added--1
['A', 'C']
{('A', 'B'): [None, None, 1, tensor([-50000.], dtype=torch.float64)], ('A', 'C'): [None, None, 1, tensor([-50000.], dtype=torch.float64)], ('B', 'C'): [None, None, 1, tensor([-50000.], dtype=torch.float64)]}
New node added--2
[0, 1, 2, 3, 4]
Allowed edges: 
('A', 'B')
Allowed edges: 
('A', 'C')
Allowed edges: 
('B', '

In [4]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_dict[(n,k)] = uid

print(uid_dict)

{(0, 3): 0, (0, 4): 1, (0, 6): 5, (1, 3): 0, (1, 5): 2, (1, 6): 3, (2, 4): 1, (2, 5): 2, (2, 6): 4, (3, 6): 4, (4, 6): 3, (5, 6): 5}


In [5]:
#Changing Initial Conditions
import networkx as nx
#Changin k_on
new_kon = [1000.0,1000.0,1000.0,1000.0,1000.0,1000.0]*np.array([1e0])  #Poor yield Simruntime = 1sec

update_kon_dict = {}
for edge in rn.network.edges:
    print(rn.network.get_edge_data(edge[0],edge[1]))
    update_kon_dict[edge] = new_kon[uid_dict[edge]]

nx.set_edge_attributes(rn.network,update_kon_dict,'k_on')

# for edge in rn.network.edges:
#     print(rn.network.get_edge_data(edge[0],edge[1]))
vec_rn = VectorizedRxnNet(rn, dev='cpu')
print(vec_rn.kon)

{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-50000.], dtype=torch.float64), 'uid': 0}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-50000.], dtype=torch.float64), 'uid': 1}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-100000.], dtype=torch.float64), 'uid': 5}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-50000.], dtype=torch.float64), 'uid': 0}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-50000.], dtype=torch.float64), 'uid': 2}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-100000.], dtype=torch.float64), 'uid': 3}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-50000.], dtype=torch.float64), 'uid': 1}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-50000.], dtype=torch.float64), 'uid': 2}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-100000.], dtype=torch.float64), 'uid': 4}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-100000.], dtype=to

## The Equilibrium Solution ##
First we will find the equilibrium solution for this system.

In [6]:
vec_rn.update_reaction_net(rn,1)
poly_system = EquilibriumSolver(rn)
solution = poly_system.solve()
if solution == None:
    print("No Equilibrium solution")
else:
    print(solution)
    print("Equilibrium expected yield: ", 100 * solution[-1] / min(vec_rn.initial_copies[:vec_rn.num_monomers]), '%')
print(vec_rn.kon)

No aceeptable solution found
No Equilibrium solution
Parameter containing:
tensor([1000., 1000., 1000., 1000., 1000., 1000.], dtype=torch.float64,
       requires_grad=True)


In [7]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_val = {'reactants':reactants,'kon':v['k_on'],'score':v['rxn_score'],'koff':v['k_off'],'uid':uid}
        if uid not in uid_dict.keys():
            uid_dict[uid] = uid_val
    print(gtostr(rn.network.nodes[n]['struct']))
    #for r_set in rn.get_reactant_sets(n):
    #    print(tuple(r_set))
    #print(rn.network[n]['struct'])
ind_sort = np.argsort(vec_rn.kon.detach().numpy())
for i in ind_sort:
    print(vec_rn.kon[i])
    print(uid_dict[i])

A
B
C
AB
AC
BC
ABC
tensor(1000., dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'A'}, {'B'}), 'kon': 999.9999999999998, 'score': tensor([-50000.], dtype=torch.float64), 'koff': 2.7423593919635816e-07, 'uid': 0}
tensor(1000., dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'A'}, {'C'}), 'kon': 999.9999999999998, 'score': tensor([-50000.], dtype=torch.float64), 'koff': 2.7423593919635816e-07, 'uid': 1}
tensor(1000., dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'B'}, {'C'}), 'kon': 999.9999999999998, 'score': tensor([-50000.], dtype=torch.float64), 'koff': 2.7423593919635816e-07, 'uid': 2}
tensor(1000., dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'B'}, {'C', 'A'}), 'kon': 999.9999999999998, 'score': tensor([-100000.], dtype=torch.float64), 'koff': 7.520535034690868e-17, 'uid': 3}
tensor(1000., dtype=torch.float64, grad_fn=<SelectBackward>)
{'reactants': ({'C'}, {'B', 'A'}), 'kon': 999.9999999999998, 'score': tensor([-1000

In [8]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
from torch import DoubleTensor as Tensor
import torch

node_map = {}
for node in rn.network.nodes():
    node_map[gtostr(rn.network.nodes[node]['struct'])] = node

print(node_map)

def get_max_edge(n):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        if len(edges)==0:
            return(False)
        kon_max = -1
        next_node = -1
        
        kon_sum = 0
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']
            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)
def get_node_flux(n):
    total_flux_outedges = 0
    total_flux_inedges = 0
    #Go over all the out edges
    edges_out = rn.network.out_edges(n)
    if len(edges_out)>0:

        for edge in edges_out:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]

            #Calculate k_off also
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Getting conc. of reactants and products
            #Get product
            prod = gtostr(rn.network.nodes[edge[1]]['struct']) 
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))

            #Net flux from this edge = Generation - consumption
            edge_flux = koff*solution[edge[1]] - temp_kon*(solution[edge[0]])*(solution[node_map[react]])
            #edge_flux = koff*vec_rn.copies_vec[edge[1]] 

            print("Reaction: ", gtostr(rn.network.nodes[edge[0]]['struct']), "+",react," -> ",prod)
            print("Net flux: ",edge_flux)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Free Energy: ",vec_rn.rxn_score_vec[uid])
            print("Reaction data OUTWARD: ")
            print(data)

            total_flux_outedges+=edge_flux
    
    #Now go over all the in edges
    edges_in = rn.network.in_edges(n)
    react_list = []
    if len(edges_in) > 0:
        for edge in edges_in:
            if edge[0] in react_list:
                continue
            data = rn.network.get_edge_data(edge[0],edge[1])
            uid = data['uid']


            #Get generation rates; which would be kon
            temp_kon = vec_rn.kon[uid]

            #Get consumption rates; which is k_off
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Get conc. of reactants and products
            prod = gtostr(rn.network.nodes[edge[1]]['struct'])
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))
            react_list.append(node_map[react])
            #Net flux from this edge = Generation - consumption
            edge_flux_in = temp_kon*(solution[edge[0]])*(solution[node_map[react]])- koff*solution[edge[1]]
            #edge_flux_in = koff*vec_rn.copies_vec[edge[1]]
            


            print("Reaction: ", prod ," -> ",gtostr(rn.network.nodes[edge[0]]['struct']), "+",react)
            print("Net flux: ",edge_flux_in)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Raction data INWARD: ")
            print(data)

            total_flux_inedges+=edge_flux_in
    net_node_flux = total_flux_outedges + total_flux_inedges
    
    return(net_node_flux)
pathway = []
kon_sumarray = []
total_con_rate = {}
net_flux = {}

if solution != None:
    for n in rn.network.nodes():

        n_str = gtostr(rn.network.nodes[n]['struct']) 

        paths = [n_str]
        kon_sum = 0
        temp_node = n
        max_edge = True
        consumption_rate = 0
        if n < len(rn.network.nodes()):#num_monomers:
    #         print("Current node: ")
    #         print(n_str)
            while max_edge:
                max_edge = get_max_edge(temp_node)
                if max_edge:
                    total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                    temp_node = max_edge[1]
                    kon_sum += max_edge[0].item()


    #                 print("Next node: ")
    #                 print(temp_node)

                    paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
                else:
                    break
            pathway.append(paths)
            kon_sumarray.append(kon_sum)
            paths=[]

        print("-------------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------------")
        print("|                                                                             |")
        node_flux = get_node_flux(n)
        net_flux[gtostr(rn.network.nodes[n]['struct'])] = node_flux
        print("|                                                                             |")
        print("-------------------------------------------------------------------------------")
        print("-------------------------------------------------------------------------------")

    print(pathway)
    print(kon_sumarray)
    #print(total_con_rate)

{'A': 0, 'B': 1, 'C': 2, 'AB': 3, 'AC': 4, 'BC': 5, 'ABC': 6}


In [9]:
if solution != None:
    for k,v in sorted(total_con_rate.items(),key=lambda x : x[1]):
        print(k," : ", v.item())

In [10]:
if solution !=None:
    for k,v in sorted(net_flux.items(),key=lambda x : x[1]):
        print(k," : ", v)



## Using the optimizer with a 1 second simulation runtime ##

In [ ]:
vec_rn.reset(reset_params=True)
optim = Optimizer(reaction_network=vec_rn,
                  sim_runtime=1,
                  optim_iterations=10000,
                  learning_rate=1,
                  device='cpu',method='RMSprop')
optim.rn.update_reaction_net(rn)
optim.optimize()

Using CPU
Reaction Parameters before optimization: 
[Parameter containing:
tensor([1000., 1000., 1000., 1000., 1000., 1000.], dtype=torch.float64,
       requires_grad=True)]
Using CPU
Next time:  tensor(163297.0706, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 0 was 57.0%
current params: tensor([ 990.0148,  990.0148,  990.0145, 1009.9852, 1009.9852, 1009.9856],
       dtype=torch.float64)
Using CPU
Next time:  tensor(166308.9305, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 1 was 57.4%
current params: tensor([ 982.9075,  982.9053,  982.8997, 1017.0236, 1017.0177, 1017.0345],
       dtype=torch.float64)
Using CPU
Next time:  tensor(168498.6975, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 2 was 57.7%
current params: tensor([ 977.0740,  977.0705,  977.0597, 1022.7648, 1022.7558, 1022.7849],
       dtype=torch.float64)
Using CPU
Next time:  tensor(170325.4196, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim it

Next time:  tensor(200358.4486, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 34 was 61.3%
current params: tensor([ 890.4113,  890.4025,  890.2168, 1105.2599, 1105.2388, 1105.5396],
       dtype=torch.float64)
Using CPU
Next time:  tensor(201115.0270, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 35 was 61.4%
current params: tensor([ 888.5371,  888.5282,  888.3369, 1106.9938, 1106.9727, 1107.2816],
       dtype=torch.float64)
Using CPU
Next time:  tensor(201867.2903, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 36 was 61.4%
current params: tensor([ 886.6830,  886.6741,  886.4772, 1108.7072, 1108.6859, 1109.0030],
       dtype=torch.float64)
Using CPU
Next time:  tensor(202615.5087, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 37 was 61.5%
current params: tensor([ 884.8483,  884.8394,  884.6368, 1110.4008, 1110.3795, 1110.7046],
       dtype=torch.float64)
Using CPU
Next time:  tensor(203359.9350, 

Next time:  tensor(225452.3238, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 69 was 63.5%
current params: tensor([ 833.0840,  833.0742,  832.6907, 1157.4654, 1157.4428, 1158.0269],
       dtype=torch.float64)
Using CPU
Next time:  tensor(226152.6960, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 70 was 63.6%
current params: tensor([ 831.6172,  831.6074,  831.2182, 1158.7798, 1158.7572, 1159.3493],
       dtype=torch.float64)
Using CPU
Next time:  tensor(226853.1548, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 71 was 63.7%
current params: tensor([ 830.1570,  830.1472,  829.7523, 1160.0874, 1160.0648, 1160.6650],
       dtype=torch.float64)
Using CPU
Next time:  tensor(227553.7569, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 72 was 63.7%
current params: tensor([ 828.7030,  828.6932,  828.2926, 1161.3883, 1161.3657, 1161.9740],
       dtype=torch.float64)
Using CPU
Next time:  tensor(228254.5579, 

Next time:  tensor(250327.6736, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 104 was 65.4%
current params: tensor([ 784.7901,  784.7798,  784.1969, 1200.2385, 1200.2153, 1201.0832],
       dtype=torch.float64)
Using CPU
Next time:  tensor(251057.7706, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 105 was 65.5%
current params: tensor([ 783.4819,  783.4716,  782.8830, 1201.3833, 1201.3601, 1202.2361],
       dtype=torch.float64)
Using CPU
Next time:  tensor(251789.3676, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 106 was 65.5%
current params: tensor([ 782.1768,  782.1665,  781.5722, 1202.5248, 1202.5015, 1203.3856],
       dtype=torch.float64)
Using CPU
Next time:  tensor(252522.4948, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 107 was 65.6%
current params: tensor([ 780.8748,  780.8644,  780.2644, 1203.6629, 1203.6396, 1204.5318],
       dtype=torch.float64)
Using CPU
Next time:  tensor(253257.18

Next time:  tensor(276958.9770, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 139 was 67.2%
current params: tensor([ 740.5135,  740.5027,  739.7217, 1238.6151, 1238.5916, 1239.7439],
       dtype=torch.float64)
Using CPU
Next time:  tensor(277758.1406, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 140 was 67.2%
current params: tensor([ 739.2858,  739.2750,  738.4884, 1239.6688, 1239.6452, 1240.8057],
       dtype=torch.float64)
Using CPU
Next time:  tensor(274289.1396, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 141 was 66.6%
current params: tensor([ 737.9847,  737.9738,  737.2262, 1240.7950, 1240.7715, 1241.8737],
       dtype=torch.float64)
Using CPU
Next time:  tensor(275131.6016, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 142 was 66.7%
current params: tensor([ 736.6865,  736.6757,  735.9666, 1241.9180, 1241.8944, 1242.9395],
       dtype=torch.float64)
Using CPU
Next time:  tensor(275976.12

...

current params: tensor([  19.8336,   19.7405,  897.7837, 2610.6615, 2602.0459, 2456.3039],
       dtype=torch.float64)
Using CPU
Next time:  tensor(1.0001, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 1975 was 98.9%
current params: tensor([  19.8326,   19.7391,  898.6424, 2611.5619, 2602.9455, 2457.1352],
       dtype=torch.float64)
Using CPU
Next time:  tensor(1.0001, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 1976 was 99.0%
current params: tensor([  19.8315,   19.7377,  899.5009, 2612.4620, 2603.8450, 2457.9662],
       dtype=torch.float64)
Using CPU
Next time:  tensor(1.0001, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 1977 was 99.0%
current params: tensor([  19.8305,   19.7364,  900.3594, 2613.3621, 2604.7443, 2458.7971],
       dtype=torch.float64)
Using CPU
Next time:  tensor(1.0000, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 1978 was 99.0%
current params: tensor([  19.8295,   19.7350

Next time:  tensor(1.0001, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 2009 was 99.0%
current params: tensor([  19.7950,   19.6884,  927.7792, 2642.0865, 2633.4512, 2485.2741],
       dtype=torch.float64)
Using CPU
Next time:  tensor(1.0001, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 2010 was 99.0%
current params: tensor([  19.7938,   19.6868,  928.6347, 2642.9820, 2634.3463, 2486.0984],
       dtype=torch.float64)
Using CPU
Next time:  tensor(1.0001, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 2011 was 99.0%
current params: tensor([  19.7926,   19.6852,  929.4902, 2643.8774, 2635.2413, 2486.9226],
       dtype=torch.float64)
Using CPU
Next time:  tensor(1.0001, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 2012 was 99.0%
current params: tensor([  19.7914,   19.6836,  930.3456, 2644.7727, 2636.1362, 2487.7466],
       dtype=torch.float64)
Using CPU
Next time:  tensor(1.0000, dtype=torch.float

current params: tensor([  19.7513,   19.6295,  956.8362, 2672.4764, 2663.8311, 2513.2139],
       dtype=torch.float64)
Using CPU
Next time:  tensor(1.0001, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 2044 was 99.0%
current params: tensor([  19.7499,   19.6276,  957.6900, 2673.3686, 2664.7232, 2514.0332],
       dtype=torch.float64)
Using CPU
Next time:  tensor(1.0001, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 2045 was 99.0%
current params: tensor([  19.7485,   19.6258,  958.5438, 2674.2607, 2665.6151, 2514.8524],
       dtype=torch.float64)
Using CPU
Next time:  tensor(1.0001, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 2046 was 99.0%
current params: tensor([  19.7471,   19.6239,  959.3975, 2675.1528, 2666.5070, 2515.6714],
       dtype=torch.float64)
Using CPU
Next time:  tensor(1.0001, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 2047 was 99.0%
current params: tensor([  19.7457,   19.6220

Next time:  tensor(1.0001, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 2078 was 99.0%
current params: tensor([  19.6990,   19.5588,  986.7029, 2703.6590, 2695.0118, 2541.8176],
       dtype=torch.float64)
Using CPU
Next time:  tensor(1.0001, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 2079 was 99.0%
current params: tensor([  19.6974,   19.5566,  987.5559, 2704.5487, 2695.9016, 2542.6329],
       dtype=torch.float64)
Using CPU
Next time:  tensor(1.0001, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 2080 was 99.0%
current params: tensor([  19.6957,   19.5544,  988.4088, 2705.4384, 2696.7913, 2543.4480],
       dtype=torch.float64)
Using CPU
Next time:  tensor(1.0000, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 2081 was 99.0%
current params: tensor([  19.6941,   19.5522,  989.2617, 2706.3279, 2697.6810, 2544.2631],
       dtype=torch.float64)
Using CPU
Next time:  tensor(1.0000, dtype=torch.float

current params: tensor([  19.6398,   19.4787, 1015.6969, 2733.8775, 2725.2362, 2569.4817],
       dtype=torch.float64)
Using CPU
Next time:  tensor(1.0000, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 2113 was 99.0%
current params: tensor([  19.6379,   19.4762, 1016.5496, 2734.7654, 2726.1243, 2570.2938],
       dtype=torch.float64)
Using CPU
Next time:  tensor(1.0000, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 2114 was 99.0%
current params: tensor([  19.6361,   19.4736, 1017.4022, 2735.6532, 2727.0125, 2571.1057],
       dtype=torch.float64)
Using CPU
Next time:  tensor(1.0001, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 2115 was 99.0%
current params: tensor([  19.6342,   19.4710, 1018.2548, 2736.5410, 2727.9005, 2571.9176],
       dtype=torch.float64)
Using CPU
Next time:  tensor(1.0001, dtype=torch.float64, grad_fn=<AddBackward0>)
yield on sim iteration 2116 was 99.0%
current params: tensor([  19.6323,   19.4685

In [ ]:
print(optim.yield_per_iter[-1])

In [ ]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_val = {'reactants':reactants,'kon':v['k_on'],'score':v['rxn_score'],'koff':v['k_off'],'uid':uid}
        if uid not in uid_dict.keys():
            uid_dict[uid] = uid_val
    print(gtostr(rn.network.nodes[n]['struct']))
    #for r_set in rn.get_reactant_sets(n):
    #    print(tuple(r_set))
    #print(rn.network[n]['struct'])
ind_sort = np.argsort(vec_rn.kon.detach().numpy())
for i in ind_sort:
    print(vec_rn.kon[i])
    print(uid_dict[i])

In [ ]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
from torch import DoubleTensor as Tensor
import torch

node_map = {}
for node in rn.network.nodes():
    node_map[gtostr(rn.network.nodes[node]['struct'])] = node

print(node_map)

def get_max_edge(n):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        kon_max = -1
        next_node = -1

        kon_sum = 0
        total_flux_outedges = 0
        total_flux_inedges = 0
        if len(edges)==0:
            return(False)
            
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
             
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)

        
def get_node_flux(n):
    total_flux_outedges = 0
    total_flux_inedges = 0
    #Go over all the out edges
    edges_out = rn.network.out_edges(n)
    if len(edges_out)>0:

        for edge in edges_out:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]

            #Calculate k_off also
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Getting conc. of reactants and products
            #Get product
            prod = gtostr(rn.network.nodes[edge[1]]['struct']) 
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))

            #Net flux from this edge = Generation - consumption
            edge_flux = koff*vec_rn.copies_vec[edge[1]] - temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])
            #edge_flux = koff*vec_rn.copies_vec[edge[1]] 

            print("Reaction: ", gtostr(rn.network.nodes[edge[0]]['struct']), "+",react," -> ",prod)
            print("Net flux: ",edge_flux)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Reaction data OUTWARD: ")
            print(data)

            total_flux_outedges+=edge_flux
    
    #Now go over all the in edges
    edges_in = rn.network.in_edges(n)
    react_list = []
    if len(edges_in) > 0:
        for edge in edges_in:
            if edge[0] in react_list:
                continue
            data = rn.network.get_edge_data(edge[0],edge[1])
            uid = data['uid']


            #Get generation rates; which would be kon
            temp_kon = vec_rn.kon[uid]

            #Get consumption rates; which is k_off
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Get conc. of reactants and products
            prod = gtostr(rn.network.nodes[edge[1]]['struct'])
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))
            react_list.append(node_map[react])
            #Net flux from this edge = Generation - consumption
            edge_flux_in = temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])- koff*vec_rn.copies_vec[edge[1]]
            #edge_flux_in = koff*vec_rn.copies_vec[edge[1]]
            


            print("Reaction: ", prod ," -> ",gtostr(rn.network.nodes[edge[0]]['struct']), "+",react)
            print("Net flux: ",edge_flux_in)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Raction data INWARD: ")
            print(data)

            total_flux_inedges+=edge_flux_in
    net_node_flux = total_flux_outedges + total_flux_inedges
    
    return(net_node_flux)
    
pathway = []
kon_sumarray = []
total_con_rate = {}
net_flux = {}
for n in rn.network.nodes():
    
    n_str = gtostr(rn.network.nodes[n]['struct']) 
    
    paths = [n_str]
    kon_sum = 0
    temp_node = n
    max_edge = True
    consumption_rate = 0
    if n < len(rn.network.nodes()):#num_monomers:
#         print("Current node: ")
#         print(n_str)
        while max_edge:
            max_edge = get_max_edge(temp_node)
            if max_edge:
                total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                
                temp_node = max_edge[1]
                kon_sum += max_edge[0].item()
                
                
#                 print("Next node: ")
#                 print(temp_node)

                paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
            else:
                break
        pathway.append(paths)
        kon_sumarray.append(kon_sum)
        paths=[]
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")
    print("|                                                                             |")
    node_flux = get_node_flux(n)
    net_flux[gtostr(rn.network.nodes[n]['struct'])] = node_flux
    print("|                                                                             |")
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")

print(pathway)
print(kon_sumarray)

#print(total_con_rate)

In [ ]:
for k,v in sorted(total_con_rate.items(),key=lambda x : x[1]):
    print(k," : ", v.item())

In [ ]:
for k,v in sorted(net_flux.items(),key=lambda x : x[1]):
    print(k," : ", v)

print(vec_rn.copies_vec)

In [ ]:
for n in rn.network.nodes():
    print(gtostr(rn.network.nodes[n]['struct']), " : ", n)

Let's first visualize some of the data.

**Without any optimization**


In [ ]:
nodes_list = ['A','B','C','ABC','AB','BC','AC']
#nodes_list = ['AB','BMS','ABS','AMS','ABMS','AM','BM','MS']
optim.plot_observable(0,nodes_list)


**After 750 optimization iterations**


In [ ]:

optim.plot_observable(-1,nodes_list)


In [ ]:
optim.plot_yield()

It seems like we've found a stable solution that produces greater yield than equilibrium. This should be thermodynamically
impossible. Let's try to find an explanation. We'll run simulations using the learned optimal parameters at a few different
timescales.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
fig, ax = plt.subplots(1, 4)
optim_rn = optim.rn
for i, runtime in enumerate([1, 5, 10]):
    optim_rn.reset()
    sim = VecSim(optim_rn, runtime, device='cpu')
    y = sim.simulate()
    sim.plot_observable(nodes_list,ax=ax[i])
    ax[i].set_title("runtime: " + str(runtime) + " seconds")
fig.set_size_inches(18, 6)
plt.show()

In [ ]:
def get_max_edge(n):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        kon_max = -1
        next_node = -1

        kon_sum = 0
        total_flux_outedges = 0
        total_flux_inedges = 0
        if len(edges)==0:
            return(False)
            
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
             
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)

        
def get_node_flux(n):
    total_flux_outedges = 0
    total_flux_inedges = 0
    #Go over all the out edges
    edges_out = rn.network.out_edges(n)
    if len(edges_out)>0:

        for edge in edges_out:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]

            #Calculate k_off also
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Getting conc. of reactants and products
            #Get product
            prod = gtostr(rn.network.nodes[edge[1]]['struct']) 
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))

            #Net flux from this edge = Generation - consumption
            edge_flux = koff*vec_rn.copies_vec[edge[1]] - temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])
            #edge_flux = koff*vec_rn.copies_vec[edge[1]] 

            print("Reaction: ", gtostr(rn.network.nodes[edge[0]]['struct']), "+",react," -> ",prod)
            print("Net flux: ",edge_flux)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Reaction data OUTWARD: ")
            print(data)

            total_flux_outedges+=edge_flux
    
    #Now go over all the in edges
    edges_in = rn.network.in_edges(n)
    react_list = []
    if len(edges_in) > 0:
        for edge in edges_in:
            if edge[0] in react_list:
                continue
            data = rn.network.get_edge_data(edge[0],edge[1])
            uid = data['uid']


            #Get generation rates; which would be kon
            temp_kon = vec_rn.kon[uid]

            #Get consumption rates; which is k_off
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Get conc. of reactants and products
            prod = gtostr(rn.network.nodes[edge[1]]['struct'])
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))
            react_list.append(node_map[react])
            #Net flux from this edge = Generation - consumption
            edge_flux_in = temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])- koff*vec_rn.copies_vec[edge[1]]
            #edge_flux_in = koff*vec_rn.copies_vec[edge[1]]
            


            print("Reaction: ", prod ," -> ",gtostr(rn.network.nodes[edge[0]]['struct']), "+",react)
            print("Net flux: ",edge_flux_in)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Raction data INWARD: ")
            print(data)

            total_flux_inedges+=edge_flux_in
    net_node_flux = total_flux_outedges + total_flux_inedges
    
    return(net_node_flux)
    
pathway = []
kon_sumarray = []
total_con_rate = {}
net_flux = {}
for n in rn.network.nodes():
    
    n_str = gtostr(rn.network.nodes[n]['struct']) 
    
    paths = [n_str]
    kon_sum = 0
    temp_node = n
    max_edge = True
    consumption_rate = 0
    if n < len(rn.network.nodes()):#num_monomers:
#         print("Current node: ")
#         print(n_str)
        while max_edge:
            max_edge = get_max_edge(temp_node)
            if max_edge:
                total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                
                temp_node = max_edge[1]
                kon_sum += max_edge[0].item()
                
                
#                 print("Next node: ")
#                 print(temp_node)

                paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
            else:
                break
        pathway.append(paths)
        kon_sumarray.append(kon_sum)
        paths=[]
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")
    print("|                                                                             |")
    node_flux = get_node_flux(n)
    net_flux[gtostr(rn.network.nodes[n]['struct'])] = node_flux
    print("|                                                                             |")
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")

print(pathway)
print(kon_sumarray)

#print(total_con_rate)

In [ ]:
for k,v in sorted(net_flux.items(),key=lambda x : x[1]):
    print(k," : ", v)

print(vec_rn.copies_vec)
print(vec_rn.kon)

In [ ]:
print(solution)
poly_system = EquilibriumSolver(rn)
solution = poly_system.solve(init_val=vec_rn.copies_vec.detach().numpy().tolist())
#solution = poly_system.solve(verifyBool = False)
if solution == None:
    print("No Equilibrium solution")
else:
    print(solution)
    print("Equilibrium expected yield: ", 100 * solution[-1] / min(vec_rn.initial_copies[:vec_rn.num_monomers]), '%')
print(vec_rn.kon)

In [ ]:
print(solution)
poly_system = EquilibriumSolver(rn)
solution = poly_system.solve(verifyBool = False)
if solution == None:
    print("No Equilibrium solution")
else:
    print(solution)
    print("Equilibrium expected yield: ", 100 * solution[-1] / min(vec_rn.initial_copies[:vec_rn.num_monomers]), '%')
print(vec_rn.kon)

Clearly, the equilibrium reached by the system still matches the equilibrium solution. We have however found a set of parameters that can increase available complete AP2 at some point before equilibrium to levels significantly higher than at equilibrium. We don't observe any trapping, but have uncovered an interesting effect. 

Now we'll move on to looking at ARP23. This is 7 subunits, which drastically increases the number of possible reactions. Expect longer runtimes. 